# Baseline Static PCA and kSVD

Can't do dynamic PCA/kSVD because each subject has at least 124 points of 34716 features,<br>
and you can't enforce a notion of temporal proximity with these methods

- Rest, nback, emoid individually (train codebook on 400)
- and rest+nback+emoid together (train codebook on 200/200/200, same subjects from each)

Response variables are are 

- age, 
- sex, 
- wrat

Evaluation methods are

- Ridge regression
- Elastic MLP
- LatSim

Number of components are 10,50,100,200,300,400,500,600,800 (+ 1000,1200,1400,1600 for kSVD only)

In [1]:
# Using newly preprocessed subjects

import pickle

metadictname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_agesexwrat.pkl'
alltsname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_PowerTS_float2.pkl'

with open(metadictname, 'rb') as f:
    metadict = pickle.load(f)

with open(alltsname, 'rb') as f:
    allts = pickle.load(f)
    
print(list(metadict.keys()))
print(list(allts.keys()))
print('Complete')

['age', 'sex', 'wrat', 'missingage', 'missingsex', 'missingwrat', 'failedqc']
['emoid', 'nback', 'rest']
Complete


In [2]:
# Using newly preprocessed subjects

import pickle

metadictname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_agesexwrat.pkl'
alltsname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_PowerTS_float2.pkl'

with open(metadictname, 'rb') as f:
    metadict = pickle.load(f)

with open(alltsname, 'rb') as f:
    allts = pickle.load(f)
    
print(list(metadict.keys()))
print(list(allts.keys()))
print('Complete')

['age', 'sex', 'wrat', 'missingage', 'missingsex', 'missingwrat', 'failedqc']
['emoid', 'nback', 'rest']
Complete


In [3]:
'''
Get subjects that have all tasks and paras specified
Functions for creating independent and response variables
'''

import numpy as np

def get_subs(allts, metadict, tasks, paras):
    # Get subs for all paras
    for i,para in enumerate(paras):
        tmpset = set([int(sub[4:]) for sub in allts[para].keys()])
        if i == 0:
            paraset = tmpset
        else:
            paraset = paraset.intersection(tmpset)
    # Get subs for all tasks
    for i,task in enumerate(tasks):
        tmpset = set([sub for sub in metadict[task].keys()])
        if i == 0:
            taskset = tmpset
        else:
            taskset = paraset.intersection(tmpset)
    # Remove QC failures
    allsubs = taskset.intersection(paraset)
    for badsub in metadict['failedqc']:
        try:
            allsubs.remove(int(badsub[4:]))
        except:
            pass
    return allsubs

def get_X(allts, paras, subs):
    X = []
    for para in paras:
        pX = [allts[para][f'sub-{sub}'] for sub in subs]
        pX = np.stack(pX)
        X.append(pX)
    return X

def get_y(metadict, tasks, subs):
    y = []
    for task in tasks:
        if task == 'age' or task == 'wrat':
            var = [metadict[task][sub] for sub in subs]
            var = np.array(var)
            y.append(var)
        if task == 'sex':
            maleness = [metadict[task][sub] == 'M' for sub in subs]
            maleness = np.array(maleness)
            sex = np.stack([maleness, 1-maleness], axis=1)
            y.append(sex)
    return y

subs = get_subs(allts, metadict, ['age'], ['rest', 'nback', 'emoid'])
print(len(subs))

X = get_X(allts, ['rest', 'nback', 'emoid'], subs)
print(X[0].shape)

847
(847, 264, 124)


In [4]:
# TS to condensed FC

from scipy import signal

def butter_bandpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = [cutoff[0] / nyq, cutoff[1] / nyq]
    b, a = signal.butter(order, normal_cutoff, btype='band', analog=False)
    return b, a

def butter_bandpass_filter(data, cutoff, fs, order=5):
    b, a = butter_bandpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

tr = 1.83

def filter_design_ts(X):
    Xs = []
    for i in range(X.shape[0]):
        nX = butter_bandpass_filter(X[i], [0.01, 0.2], 1/tr)
        Xs.append(nX)
    return np.stack(Xs)

def ts_to_flat_fc(X):
    p = np.corrcoef(X)
    a,b = np.triu_indices(p[0].shape[0], 1)
    p = p[a,b]
    return p

p = [np.stack([ts_to_flat_fc(ts) for ts in filter_design_ts(Xp)]) for Xp in X]
print(p[0].shape)

(847, 34716)


In [5]:
import sys

sys.path.append('../../LatentSimilarity')

from latsim import LatSim

print('Complete')

Complete


In [12]:
from dask_ml.decomposition import PCA
import dask.array as da
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, ncodes):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(ncodes, 40).float().cuda()
        self.l2 = nn.Linear(40,1).float().cuda()
        
    def train(self, xtr, ytr, nepochs=1000, lr=1e-1, l1=1e-1, l2=1e-4, pperiod=100, verbose=False):
        optim = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=l2)
        sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)
        
        for epoch in range(nepochs):
            optim.zero_grad()
            yhat = self(xtr)
            loss = mseLoss(yhat, ytr)**0.5
            l1loss = l1*torch.sum(torch.abs(self.l1.weight))
            (loss+l1loss).backward()
            optim.step()
            sched.step(loss)
            if verbose:
                if epoch % pperiod == 0 or epoch == nepochs-1:
                    print(f'{epoch} {[float(l) for l in [loss, l1loss]]} {sched._last_lr}')
                    
    def predict(self, xt, yt):
        with torch.no_grad():
            return mseLoss(self(xt), yt)**0.5
                    
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x).squeeze()
        return x

ncodesall = [10,50,100,200,300,400,500,600,800]
ntrainall = [400,400,400,400,400,400,500,600,800]

modstr = 'nback'
modidx = 1

for ncodes,ntrain in zip(ncodesall, ntrainall):
    for split in range(3):
        idcs = np.arange(p[modidx].shape[0])
        np.random.shuffle(idcs)
        t0 = time.time()
        dx = da.from_array(p[modidx], chunks=(1000,1000))
        pca = PCA(n_components=ncodes)
        pca.fit(dx[idcs[:ntrain]])
        z = pca.transform(dx)
        z = z.persist()
        t1 = time.time()
        print(z.shape)
        print(f'pca time {t1-t0}')

        nreps = 10
        trainsizes = [30,50,100,200,300,400,500,600,700,800]
        res = np.zeros((nreps,len(trainsizes)))
        l2 = 1e0

        for rep in range(nreps):
            losses = []

            idcs = np.arange(z.shape[0])
            np.random.shuffle(idcs)

            for ntrain in trainsizes:
                xps = torch.from_numpy(np.asarray(z)).float().cuda()
                xps = torch.cat([xps, torch.ones(xps.shape[0], 1).float().cuda()], dim=1)
                xtr = xps[idcs[:ntrain]]
                xt = xps[idcs[ntrain:]]

                y = get_y(metadict, ['age'], subs)[0]
                y_t = torch.from_numpy(y).float().cuda()
                ytr = y_t[idcs[:ntrain]]
                yt = y_t[idcs[ntrain:]]

                # REDUCE THIS TO GET GOOD RESULTS WITH SPARSITY 0.01->0.001 or 0.0001
                w, _, _, _ = torch.linalg.lstsq(xtr.T@xtr + l2*torch.eye(ncodes+1).float().cuda(), xtr.T@ytr)

                print(torch.mean((yt-xt@w)**2)**0.5)
                losses.append(float(torch.mean((yt-xt@w)**2)**0.5))

            print(f'Finished {rep}')
            res[rep,:] = losses

        print(np.mean(res, axis=0))
        print(np.std(res, axis=0))
            
        print('Finished lstsq')

        with open(f'/home/anton/Documents/Tulane/Research/Work/LatSimEC2/DictEst/PCA/{modstr}-lstsq-mean.csv', 'a') as f:
            f.write(f'{ncodes},{split},{",".join([str(val) for val in np.mean(res, axis=0)])}\n')

        with open(f'/home/anton/Documents/Tulane/Research/Work/LatSimEC2/DictEst/PCA/{modstr}-lstsq-std.csv', 'a') as f:
            f.write(f'{ncodes},{split},{",".join([str(val) for val in np.std(res, axis=0)])}\n')
            
        mseLoss = nn.MSELoss()

        nreps = 10
        trainsizes = [30,50,100,200,300,400,500,600,700,800]
        res = np.zeros((nreps,len(trainsizes)))

        for rep in range(nreps):

            idcs = np.arange(z.shape[0])
            np.random.shuffle(idcs)

            losses = []

            for ntrain in trainsizes:
                xps = torch.from_numpy(np.asarray(z)).float().cuda()
                xtr = xps[idcs[:ntrain]]
                xt = xps[idcs[ntrain:]]

                y = get_y(metadict, ['age'], subs)[0]
                y_t = torch.from_numpy(y).float().cuda()
                ytr = y_t[idcs[:ntrain]]
                yt = y_t[idcs[ntrain:]]

                mlp = MLP(ncodes)
                # 1e-3 good for age 1e-2 good for wrat
                mlp.train(xtr, ytr, lr=1e-2, nepochs=1000, l1=1e0, l2=1e-3)
                loss = mlp.predict(xt, yt)

                losses.append(float(loss))
                print(float(loss))

            res[rep,:] = losses
            print(f'Finished {rep}')

        print(np.mean(res, axis=0))
        print(np.std(res, axis=0))
        
        with open(f'/home/anton/Documents/Tulane/Research/Work/LatSimEC2/DictEst/PCA/{modstr}-mlp-mean.csv', 'a') as f:
            f.write(f'{ncodes},{split},{",".join([str(val) for val in np.mean(res, axis=0)])}\n')

        with open(f'/home/anton/Documents/Tulane/Research/Work/LatSimEC2/DictEst/PCA/{modstr}-mlp-std.csv', 'a') as f:
            f.write(f'{ncodes},{split},{",".join([str(val) for val in np.std(res, axis=0)])}\n')
            
        print('Finished MLP')
        
        nreps = 10
        trainsizes = [30,50,100,200,300,400,500,600,700,800]
        res = np.zeros((nreps,len(trainsizes)))

        nepochs = 500
        pperiod = 100
        verbose = False

        for rep in range(nreps):

            idcs = np.arange(z.shape[0])
            np.random.shuffle(idcs)

            losses = []

            for ntrain in trainsizes:
                xps = torch.from_numpy(np.asarray(z))[idcs].unsqueeze(1).float().cuda()

                mu = torch.mean(xps[:ntrain], dim=0, keepdims=True)
                std = torch.std(xps[:ntrain], dim=0, keepdims=True)
                xps = (xps-mu)/std

                xtr = xps[:ntrain]
                xt = xps[ntrain:]

                y = get_y(metadict, ['age'], subs)[0]
                y_t = torch.from_numpy(y[idcs]).float().cuda()
                ytr = y_t[:ntrain]
                yt = y_t[ntrain:]

                # dp=0.5 for wrat
                sim = LatSim(1, xps, dp=0.5, edp=0.1, wInit=1e-4, dim=10, temp=1)
                optim = torch.optim.Adam(sim.parameters(), lr=1e-3, weight_decay=1e-3)
                sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)

                for epoch in range(nepochs):
                    optim.zero_grad()
                    yhat = sim(xtr, [ytr])[0][0]
                    loss = mseLoss(yhat, ytr)**0.5
                    loss.backward()
                    optim.step()
                    sched.step(loss)
                    if verbose:
                        if epoch % pperiod == 0 or epoch == nepochs-1:
                            print(f'{epoch} {float(loss)} {sched._last_lr}')

                sim.eval()
                yhat = sim(xps, [y_t], np.arange(ntrain,idcs.shape[0]))[0][0][ntrain:]
                loss = mseLoss(yhat, yt)**0.5
                losses.append(float(loss))

                print(float(loss))

            res[rep,:] = losses
            print(f'Finished {rep}')

        print(np.mean(res, axis=0))
        print(np.std(res, axis=0))
        
        print('Finished latsim')
        
        with open(f'/home/anton/Documents/Tulane/Research/Work/LatSimEC2/DictEst/PCA/{modstr}-latsim-mean.csv', 'a') as f:
            f.write(f'{ncodes},{split},{",".join([str(val) for val in np.mean(res, axis=0)])}\n')

        with open(f'/home/anton/Documents/Tulane/Research/Work/LatSimEC2/DictEst/PCA/{modstr}-latsim-std.csv', 'a') as f:
            f.write(f'{ncodes},{split},{",".join([str(val) for val in np.std(res, axis=0)])}\n')

(847, 10)
pca time 0.6283316612243652
tensor(37.9263, device='cuda:0')
tensor(37.8268, device='cuda:0')
tensor(35.8202, device='cuda:0')
tensor(34.8596, device='cuda:0')
tensor(34.8266, device='cuda:0')
tensor(34.6793, device='cuda:0')
tensor(34.3288, device='cuda:0')
tensor(33.5522, device='cuda:0')
tensor(33.8064, device='cuda:0')
tensor(34.5195, device='cuda:0')
Finished 0
tensor(57.2830, device='cuda:0')
tensor(38.2530, device='cuda:0')
tensor(35.8462, device='cuda:0')
tensor(35.1462, device='cuda:0')
tensor(34.2441, device='cuda:0')
tensor(34.2634, device='cuda:0')
tensor(33.9425, device='cuda:0')
tensor(32.7267, device='cuda:0')
tensor(33.0702, device='cuda:0')
tensor(28.8153, device='cuda:0')
Finished 1
tensor(42.7748, device='cuda:0')
tensor(39.2168, device='cuda:0')
tensor(36.4993, device='cuda:0')
tensor(35.1403, device='cuda:0')
tensor(33.4259, device='cuda:0')
tensor(33.5454, device='cuda:0')
tensor(34.6391, device='cuda:0')
tensor(35.0229, device='cuda:0')
tensor(33.4904, 

42.37179946899414
41.40070343017578
37.66204071044922
36.220703125
36.4393310546875
36.61768341064453
37.49641799926758
37.19482421875
37.121681213378906
36.76382827758789
Finished 0
38.39251708984375
37.31061553955078
36.86189651489258
36.147090911865234
36.37052917480469
35.72047424316406
36.28553009033203
37.24943923950195
37.414024353027344
33.61748504638672
Finished 1
43.413692474365234
38.27040100097656
36.46181106567383
35.86273956298828
36.39316177368164
36.21809768676758
36.24336242675781
36.09786605834961
35.11429214477539
34.602081298828125
Finished 2
41.285221099853516
38.635467529296875
37.48316192626953
36.175750732421875
36.420433044433594
36.54480743408203
36.198123931884766
35.578773498535156
34.740596771240234
29.22987174987793
Finished 3
44.63951110839844
40.391258239746094
35.609153747558594
35.99089431762695
35.832794189453125
35.793663024902344
37.24181365966797
37.12947463989258
37.473819732666016
38.11723709106445
Finished 4
40.82905197143555
40.16706466674805
3

33.90705108642578
37.36651611328125
Finished 0
37.76063537597656
35.34185791015625
34.92770004272461
34.56296157836914
34.27622604370117
33.94938659667969
33.474700927734375
33.9802360534668
33.997318267822266
33.896419525146484
Finished 1
35.8976936340332
35.279170989990234
35.374000549316406
35.54692840576172
35.70965576171875
35.77009963989258
36.255699157714844
37.36005783081055
37.84690475463867
39.72151184082031
Finished 2
39.39600372314453
36.441226959228516
36.28809356689453
35.0756721496582
34.48692321777344
34.52702331542969
34.21171951293945
34.096229553222656
34.01908493041992
36.278236389160156
Finished 3
39.54011917114258
39.564361572265625
35.20541763305664
34.689537048339844
35.55907440185547
35.64155197143555
35.209625244140625
34.99641799926758
33.86220169067383
35.20431137084961
Finished 4
43.777652740478516
35.4852409362793
35.831504821777344
34.58084487915039
35.65517044067383
35.257286071777344
35.10378646850586
35.13479995727539
34.33916091918945
36.4752197265625

33.95950698852539
34.987762451171875
34.90336227416992
34.16618347167969
32.781028747558594
33.64877700805664
36.3570442199707
Finished 1
42.737518310546875
47.825016021728516
36.63062286376953
34.15713882446289
34.4922981262207
34.5458984375
35.387630462646484
36.32225799560547
35.88361358642578
35.49723434448242
Finished 2
36.74299621582031
47.059085845947266
35.561824798583984
35.34597396850586
34.683006286621094
34.33130645751953
34.667152404785156
34.73260498046875
37.22721862792969
33.322471618652344
Finished 3
37.86787033081055
35.24289321899414
36.428138732910156
35.639068603515625
35.791805267333984
35.91935729980469
36.07212448120117
36.27923583984375
35.27388381958008
35.99860382080078
Finished 4
44.28801345825195
44.83610153198242
34.84576416015625
34.773399353027344
34.63298797607422
34.04798126220703
34.47692108154297
34.386741638183594
33.50815200805664
31.82976722717285
Finished 5
48.75919723510742
40.7325553894043
34.07148742675781
34.48476791381836
34.37910461425781
3

31.16087532043457
Finished 1
48.99507141113281
56.14186477661133
35.45137405395508
34.450111389160156
32.606632232666016
33.35152816772461
33.63376235961914
33.84450912475586
32.98088455200195
31.290971755981445
Finished 2
49.862606048583984
47.69585037231445
36.18583297729492
35.013641357421875
34.96858596801758
34.880821228027344
35.06842041015625
34.13801193237305
34.23061752319336
34.191402435302734
Finished 3
55.078060150146484
45.41610336303711
35.897212982177734
34.725120544433594
34.61608123779297
34.77071762084961
35.045658111572266
35.78412628173828
36.498817443847656
38.30773162841797
Finished 4
47.21026611328125
51.253231048583984
35.859676361083984
35.10388946533203
35.37834548950195
35.156349182128906
36.39100646972656
35.07577133178711
34.90083312988281
33.621429443359375
Finished 5
48.382904052734375
37.149078369140625
36.27615737915039
35.5344352722168
35.21177673339844
34.73312759399414
35.08377456665039
35.6439208984375
35.632545471191406
39.39619064331055
Finished 6

...

45.6566047668457
42.22523880004883
42.48656463623047
35.06238555908203
33.765052795410156
32.64841842651367
33.09898376464844
31.19461441040039
33.83585739135742
34.24714660644531
Finished 9
[43.03368034 40.99939003 37.44253922 33.64237022 33.05905304 33.14444466
 32.86989746 32.97947826 33.02269497 33.24976101]
[3.85694381 3.62833801 2.76819907 1.27405438 1.11445026 0.40461068
 1.57297395 1.97894875 1.49691911 2.21445435]
Finished MLP
43.5981559753418
44.270503997802734
41.47275924682617
38.63943862915039
38.22318649291992
37.08938217163086
35.65105438232422
34.68366241455078
34.43337631225586
36.07014083862305
Finished 0
39.80668640136719
40.331214904785156
39.28978729248047
39.24348449707031
38.39070129394531
38.00065612792969
36.58225631713867
36.295013427734375
36.86893844604492
39.18180465698242
Finished 1
41.4872932434082
39.97265625
39.37944030761719
39.27244186401367
38.15134048461914
36.50900650024414
35.18560791015625
34.71820068359375
33.704559326171875
31.207639694213867
F

33.11174774169922
33.45364761352539
32.913612365722656
32.96014404296875
32.45866775512695
31.094507217407227
29.707761764526367
Finished 9
[43.11848488 41.11992607 38.69965248 33.23265648 33.19840736 33.34099426
 33.35421009 33.12567768 32.8829977  32.27597809]
[4.18216176 2.50436149 2.44670915 0.58326873 0.50400538 0.45571024
 0.96616556 0.85144387 1.00143242 2.00187716]
Finished MLP
40.07090377807617
41.88783264160156
40.901512145996094
39.403926849365234
37.11531448364258
37.237205505371094
35.540794372558594
33.93573760986328
32.69953918457031
29.938541412353516
Finished 0
40.58204650878906
40.845359802246094
39.85862350463867
38.84035110473633
37.93650436401367
37.337467193603516
36.93373107910156
36.87440490722656
35.93442916870117
32.79664611816406
Finished 1
44.30358123779297
40.72856140136719
39.483272552490234
38.64803695678711
37.22831344604492
37.976436614990234
36.639991760253906
36.344200134277344
35.57826232910156
37.34098434448242
Finished 2
40.50767517089844
40.094135

32.9483528137207
33.48239517211914
33.658660888671875
34.668277740478516
35.19059753417969
37.51276779174805
33.28475570678711
Finished 9
[42.08094826 40.43197861 37.21483955 33.98474236 33.13902512 32.69228363
 33.83581581 34.15932713 34.98590775 35.18082142]
[2.93768493 2.0369953  2.5720643  1.06949115 1.15971916 1.58466549
 0.92505652 1.19139186 1.72561201 2.56441397]
Finished MLP
42.61964797973633
42.18342208862305
40.174808502197266
39.914276123046875
39.40140151977539
39.271244049072266
39.1735725402832
38.227996826171875
37.80756378173828
42.00990676879883
Finished 0
43.0195426940918
41.727691650390625
41.65251541137695
39.49130630493164
39.77397155761719
39.181541442871094
39.1422004699707
39.02241134643555
39.46809387207031
38.818115234375
Finished 1
42.052642822265625
43.40633010864258
40.471378326416016
39.621910095214844
39.25933837890625
38.363861083984375
38.57570266723633
38.44768524169922
37.91409683227539
38.135704040527344
Finished 2
43.596920013427734
41.626121520996

33.38493347167969
30.214365005493164
32.203548431396484
32.114280700683594
Finished 9
[41.82484169 39.32133446 37.47514305 33.7649025  33.35112495 33.55593967
 33.5764122  33.57436047 33.61183414 32.78741322]
[3.10444832 2.12031923 1.70932402 0.66240624 1.34412784 1.24293391
 0.51234622 1.46635417 1.85475213 1.55658849]
Finished MLP
43.206947326660156
44.43486404418945
40.11293411254883
39.52316665649414
39.179386138916016
39.59016036987305
40.08366775512695
40.02985382080078
39.67938995361328
40.8920783996582
Finished 0
42.054115295410156
39.701454162597656
39.81737518310547
38.89049530029297
38.91826248168945
38.4442253112793
37.69044876098633
37.77715301513672
36.551246643066406
35.041969299316406
Finished 1
45.212669372558594
41.98430633544922
40.234798431396484
39.15851593017578
39.420223236083984
38.89805221557617
38.96889877319336
38.81611633300781
36.13554763793945
35.23234176635742
Finished 2
41.7333869934082
42.1378059387207
40.42927169799805
40.452171325683594
40.31092834472

35.75988006591797
34.14713668823242
Finished 9
[42.63692322 40.14744911 38.15573654 34.19704437 33.52155094 33.80009918
 34.0032589  33.74730873 33.70248547 34.03371525]
[3.22666319 3.79794473 1.64952376 0.91097624 1.53098624 0.72394156
 0.96325037 1.75355351 1.39451194 1.83429106]
Finished MLP
44.05780029296875
43.7926025390625
40.421321868896484
41.379608154296875
41.11030960083008
40.2154541015625
40.54966735839844
40.10356903076172
38.566444396972656
38.127769470214844
Finished 0
45.31583023071289
43.026885986328125
41.18156814575195
40.37916946411133
40.90436935424805
40.8245735168457
39.36403274536133
40.40104675292969
39.49541091918945
43.42218017578125
Finished 1
41.41217803955078
40.94091796875
40.46879577636719
39.573848724365234
40.02115249633789
40.410640716552734
39.45848846435547
39.06258010864258
40.12784957885742
41.621219635009766
Finished 2
41.450496673583984
40.28133773803711
40.748905181884766
40.294830322265625
38.9629020690918
38.699928283691406
38.961612701416016

In [33]:
import torch

nreps = 10
trainsizes = [30,50,100,200,300,400,500,600,700,800]
res = np.zeros((nreps,len(trainsizes)))
l2 = 1e0

for rep in range(nreps):
    losses = []

    idcs = np.arange(z.shape[0])
    np.random.shuffle(idcs)

    for ntrain in trainsizes:
        xps = torch.from_numpy(np.asarray(z)).float().cuda()
        xps = torch.cat([xps, torch.ones(xps.shape[0], 1).float().cuda()], dim=1)
        xtr = xps[idcs[:ntrain]]
        xt = xps[idcs[ntrain:]]

        y = get_y(metadict, ['age'], subs)[0]
        y_t = torch.from_numpy(y).float().cuda()
        ytr = y_t[idcs[:ntrain]]
        yt = y_t[idcs[ntrain:]]

        # REDUCE THIS TO GET GOOD RESULTS WITH SPARSITY 0.01->0.001 or 0.0001
        w, _, _, _ = torch.linalg.lstsq(xtr.T@xtr + l2*torch.eye(ncodes+1).float().cuda(), xtr.T@ytr)

        print(torch.mean((yt-xt@w)**2)**0.5)
        losses.append(float(torch.mean((yt-xt@w)**2)**0.5))
            
    print(f'Finished {rep}')
    res[rep,:] = losses
    
print(np.mean(res, axis=0))
print(np.std(res, axis=0))

tensor(192.5408, device='cuda:0')
tensor(195.0819, device='cuda:0')
tensor(198.7012, device='cuda:0')
tensor(205.3038, device='cuda:0')
tensor(201.2672, device='cuda:0')
tensor(185.4756, device='cuda:0')
tensor(171.2626, device='cuda:0')
tensor(147.4945, device='cuda:0')
tensor(138.3634, device='cuda:0')
tensor(185.4845, device='cuda:0')
Finished 0
tensor(193.3747, device='cuda:0')
tensor(195.6248, device='cuda:0')
tensor(201.3620, device='cuda:0')
tensor(203.9545, device='cuda:0')
tensor(196.3075, device='cuda:0')
tensor(189.9360, device='cuda:0')
tensor(176.0122, device='cuda:0')
tensor(165.2872, device='cuda:0')
tensor(135.5327, device='cuda:0')
tensor(211.7391, device='cuda:0')
Finished 1
tensor(190.9630, device='cuda:0')
tensor(196.9132, device='cuda:0')
tensor(198.1187, device='cuda:0')
tensor(199.1819, device='cuda:0')
tensor(188.6399, device='cuda:0')
tensor(180.3952, device='cuda:0')
tensor(170.5749, device='cuda:0')
tensor(160.5892, device='cuda:0')
tensor(150.4752, device='c

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, ncodes):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(ncodes, 40).float().cuda()
        self.l2 = nn.Linear(40,1).float().cuda()
        
    def train(self, xtr, ytr, nepochs=1000, lr=1e-1, l1=1e-1, l2=1e-4, pperiod=100, verbose=False):
        optim = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=l2)
        sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)
        
        for epoch in range(nepochs):
            optim.zero_grad()
            yhat = self(xtr)
            loss = mseLoss(yhat, ytr)**0.5
            l1loss = l1*torch.sum(torch.abs(self.l1.weight))
            (loss+l1loss).backward()
            optim.step()
            sched.step(loss)
            if verbose:
                if epoch % pperiod == 0 or epoch == nepochs-1:
                    print(f'{epoch} {[float(l) for l in [loss, l1loss]]} {sched._last_lr}')
                    
    def predict(self, xt, yt):
        with torch.no_grad():
            return mseLoss(self(xt), yt)**0.5
                    
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x).squeeze()
        return x
    
mseLoss = nn.MSELoss()

nreps = 10
trainsizes = [30,50,100,200,300,400,500,600,700,800]
res = np.zeros((nreps,len(trainsizes)))

for rep in range(nreps):

    idcs = np.arange(z.shape[0])
    np.random.shuffle(idcs)
    
    losses = []

    for ntrain in trainsizes:
        xps = torch.from_numpy(np.asarray(z)).float().cuda()
        xtr = xps[idcs[:ntrain]]
        xt = xps[idcs[ntrain:]]

        y = get_y(metadict, ['age'], subs)[0]
        y_t = torch.from_numpy(y).float().cuda()
        ytr = y_t[idcs[:ntrain]]
        yt = y_t[idcs[ntrain:]]

        mlp = MLP(ncodes)
        # 1e-3 good for age 1e-2 good for wrat
        mlp.train(xtr, ytr, lr=1e-2, nepochs=1000, l1=1e0, l2=1e-3)
        loss = mlp.predict(xt, yt)

        losses.append(float(loss))
        print(float(loss))
    
    res[rep,:] = losses
    print(f'Finished {rep}')
    
print(np.mean(res, axis=0))
print(np.std(res, axis=0))

40.60023880004883
38.32298278808594
39.800872802734375
34.742130279541016
34.226829528808594
34.612998962402344
34.246883392333984
32.39553451538086
32.25904846191406
33.38909912109375
Finished 0
44.191650390625
41.63274002075195
42.74654769897461
34.2310676574707
33.71150588989258
33.66259002685547
34.705528259277344
33.93981170654297
32.8906364440918
33.65440368652344
Finished 1
44.42479705810547
40.79983139038086
34.38186264038086
34.54288101196289
33.875152587890625
34.27886962890625
35.36629104614258
36.01179122924805
35.041481018066406
33.839202880859375
Finished 2
39.102012634277344
38.587257385253906
39.901031494140625
34.06235122680664
33.89718246459961
33.865177154541016
35.077754974365234
33.87831115722656
32.8411750793457
31.112560272216797
Finished 3
40.39668273925781
40.18560028076172
41.08665084838867
34.97288131713867
35.36097717285156
34.74446487426758
35.354835510253906
34.93059539794922
34.95436477661133
39.72813034057617
Finished 4
51.14301300048828
45.7835502624511

In [34]:
nreps = 10
trainsizes = [30,50,100,200,300,400,500,600,700,800]
res = np.zeros((nreps,len(trainsizes)))

nepochs = 500
pperiod = 100
verbose = False

for rep in range(nreps):

    idcs = np.arange(z.shape[0])
    np.random.shuffle(idcs)
    
    losses = []

    for ntrain in trainsizes:
        xps = torch.from_numpy(np.asarray(z))[idcs].unsqueeze(1).float().cuda()
        
        mu = torch.mean(xps[:ntrain], dim=0, keepdims=True)
        std = torch.std(xps[:ntrain], dim=0, keepdims=True)
        xps = (xps-mu)/std
    
        xtr = xps[:ntrain]
        xt = xps[ntrain:]

        y = get_y(metadict, ['age'], subs)[0]
        y_t = torch.from_numpy(y[idcs]).float().cuda()
        ytr = y_t[:ntrain]
        yt = y_t[ntrain:]

        # dp=0.5 for wrat
        sim = LatSim(1, xps, dp=0.5, edp=0.1, wInit=1e-4, dim=10, temp=1)
        optim = torch.optim.Adam(sim.parameters(), lr=1e-3, weight_decay=1e-3)
        sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)
        
        for epoch in range(nepochs):
            optim.zero_grad()
            yhat = sim(xtr, [ytr])[0][0]
            loss = mseLoss(yhat, ytr)**0.5
            loss.backward()
            optim.step()
            sched.step(loss)
            if verbose:
                if epoch % pperiod == 0 or epoch == nepochs-1:
                    print(f'{epoch} {float(loss)} {sched._last_lr}')
                    
        sim.eval()
        yhat = sim(xps, [y_t], np.arange(ntrain,idcs.shape[0]))[0][0][ntrain:]
        loss = mseLoss(yhat, yt)**0.5
        losses.append(float(loss))
        
        print(float(loss))
    
    res[rep,:] = losses
    print(f'Finished {rep}')
    
print(np.mean(res, axis=0))
print(np.std(res, axis=0))

41.362892150878906
44.08783721923828
40.386775970458984
39.970916748046875
39.931514739990234
39.34151077270508
39.55750274658203
40.0809440612793
38.64273452758789
40.01464080810547
Finished 0
44.839176177978516
41.674415588378906
40.362648010253906
40.96070098876953
40.21275329589844
39.955753326416016
39.49416732788086
41.14554214477539
41.157047271728516
38.1065788269043
Finished 1
43.523868560791016
40.72843933105469
41.81019592285156
40.60601043701172
40.59677505493164
39.57193374633789
40.452720642089844
40.665374755859375
38.41239929199219
36.44091033935547
Finished 2
44.02684020996094
42.21769332885742
40.772911071777344
39.84599685668945
39.833011627197266
40.07791519165039
40.349483489990234
39.99162673950195
40.3017578125
39.48772430419922
Finished 3
41.49198532104492
40.80345916748047
41.2951545715332
40.10832977294922
39.39784240722656
38.248497009277344
37.999969482421875
38.43232727050781
37.57269287109375
38.61994171142578
Finished 4
41.14785385131836
40.57553482055664